In [22]:
import pandas as pd
import numpy as np
import re
import glob
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [6]:
path = "../stocks/raw_data_candles/*.csv"

In [7]:
scaler = MinMaxScaler()
dfs = []
dfs_names = []
for file in glob.glob(path):
    # extract the file name without the extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    dfs_names.append(file_name)
    # read the CSV file into a dataframe
    df = pd.read_csv(file)
    
    
    #preprocess:
    df = df.drop(columns="symbol")
    df["time"] = pd.to_datetime(df["time"])
    df.index = df.pop('time')
    
    # creat list:
    dfs.append(df)
    


In [30]:
dfs

[                        open     high      low    close  volume
 time                                                           
 2023-02-01 09:59:00  79.7600  79.7600  79.5000  79.5750  394667
 2023-02-01 10:00:00  79.5900  79.6591  79.4800  79.6550  373402
 2023-02-01 10:01:00  79.6700  79.9600  79.4800  79.8800  616572
 2023-02-01 10:02:00  79.8800  80.3000  79.8600  80.0499  854847
 2023-02-01 10:03:00  80.0440  80.0440  79.5460  79.8343  565500
 ...                      ...      ...      ...      ...     ...
 2023-02-03 19:55:00  85.9599  85.9600  85.9599  85.9600    2604
 2023-02-03 19:56:00  85.9600  85.9600  85.9600  85.9600     119
 2023-02-03 19:58:00  85.9600  85.9600  85.9600  85.9600     236
 2023-02-03 19:59:00  86.0000  86.0000  86.0000  86.0000     566
 2023-02-03 20:00:00  85.9600  85.9600  85.9600  85.9600     628
 
 [2167 rows x 5 columns],
                            open        high         low       close  volume
 time                                             

In [12]:
data = pd.concat(dfs)
data

,open,high,low,close,volume
time,,,,,
2023-02-01 09:59:00,79.760,79.7600,79.500,79.5750,394667
2023-02-01 10:00:00,79.590,79.6591,79.480,79.6550,373402
2023-02-01 10:01:00,79.670,79.9600,79.480,79.8800,616572
2023-02-01 10:02:00,79.880,80.3000,79.860,80.0499,854847
2023-02-01 10:03:00,80.044,80.0440,79.546,79.8343,565500
...,...,...,...,...,...
2023-02-03 19:54:00,210.900,210.9000,210.900,210.9000,203
2023-02-03 19:56:00,210.810,210.8100,210.650,210.6500,592
2023-02-03 19:57:00,210.810,210.8100,210.650,210.6500,439


In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'volume']])

In [33]:
n_steps = 360
n_features = 5

In [15]:
train_size = int(len(df) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size - n_steps:, :]

In [16]:
def create_dataset(data):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :])
        y.append(data[i, 3])
    X, y = np.array(X), np.array(y)
    return X, y

In [32]:
train_X.shape

(1128, 60, 5)

In [19]:
train_X, train_y = create_dataset(train_data)
test_X, test_y = create_dataset(test_data)

In [35]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 360, 50)           11200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 31,451
Trainable params: 31,451
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(train_X, train_y, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from

In [ ]:
for df in df:
    df = df.drop("symbol")
    


In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
def clean_cashtags(st):
    letters = re.findall('[A-Za-z]+', st)
    return ' '.join(letters)

In [ ]:
df = df.dropna(subset=["cashtags"])

In [ ]:
df["cashtags"] = df["cashtags"].apply(clean_cashtags)

In [ ]:
def del_timestamp(st):
    return st.replace("+00:00", "")

In [ ]:
df["date"] = df["date"].apply(del_timestamp)

In [ ]:
df["date"] = pd.to_datetime(df['date'])

In [ ]:
df.index = df.pop('date')

In [ ]:
df[df["cashtags"] == "MSFT"]

In [ ]:
df_stocks = pd.read_csv("AAPL.csv")
df_stocks.columns

In [ ]:
df_stocks = df_stocks[['time', 'close']]
df_stocks.info()

In [ ]:
df_stocks["time"] = pd.to_datetime(df_stocks["time"])

In [ ]:
df_stocks.index = df_stocks.pop("time")

In [ ]:
df_stocks

In [ ]:
scaller = MinMaxScaler()

df_stocks["close_scalled"] = scaller.fit_transform(df_stocks[["close"]])


In [ ]:
df_stocks

In [ ]:
df

In [ ]:
merged_df = pd.merge(df, df_stocks, left_index=True, right_index=True)

In [ ]:
merged_df

In [ ]:
merged_short = merged_df[['tweet', 'username', 'likeCount', 'replyCount',
       'retweetCount', 'sentiment', 'close']]

In [ ]:
merged_short

In [ ]:
df_stocks